In [1]:
import os
import yaml
import numpy as np

In [2]:
DESI_TEMPLATE_PATH = "../../desihub/desitarget/py/desitarget/data/targetmask.yaml"
LVM_TEMPLATE_PATH = "./MW_plane.yaml"

In [3]:
desi_template = yaml.safe_load(open(DESI_TEMPLATE_PATH))
desi_template

{'desi_mask': [['LRG', 0, 'LRG', {'obsconditions': 'DARK'}],
  ['ELG', 1, 'ELG', {'obsconditions': 'DARK|GRAY'}],
  ['QSO', 2, 'QSO', {'obsconditions': 'DARK'}],
  ['QSO_HIZ',
   4,
   'QSO selected using high-redshift Random Forest (informational bit)',
   {'obsconditions': 'DARK'}],
  ['LRG_NORTH',
   8,
   'LRG cuts tuned for Bok/Mosaic data',
   {'obsconditions': 'DARK'}],
  ['ELG_NORTH',
   9,
   'ELG cuts tuned for Bok/Mosaic data',
   {'obsconditions': 'DARK|GRAY'}],
  ['QSO_NORTH',
   10,
   'QSO cuts tuned for Bok/Mosaic data',
   {'obsconditions': 'DARK'}],
  ['LRG_SOUTH',
   16,
   'LRG cuts tuned for DECam data',
   {'obsconditions': 'DARK'}],
  ['ELG_SOUTH',
   17,
   'ELG cuts tuned for DECam data',
   {'obsconditions': 'DARK|GRAY'}],
  ['QSO_SOUTH',
   18,
   'QSO cuts tuned for DECam data',
   {'obsconditions': 'DARK'}],
  ['SKY',
   32,
   'Blank sky locations',
   {'obsconditions': 'DARK|GRAY|BRIGHT|BACKUP|TWILIGHT12|TWILIGHT18'}],
  ['STD_FAINT',
   33,
   'Standard 

In [4]:
lvm_template = yaml.safe_load(open(LVM_TEMPLATE_PATH))

In [5]:
lvm_template.keys()

dict_keys(['LMC', 'SMC', 'ORION', 'ORION_SPARSE', 'MW1', 'MW2', 'THOR', 'FULLSKY'])

In [6]:
LVM_TARGET_DESC = dict(zip(
    lvm_template.keys(),
    [
        "LMC",
        "SMC",
        "ORION",
        "sparse sampling of the surrounding orion region",
        "MW1",
        "MW2",
        "THOR",
        "sparse sampling of the full sky"
    ]))
LVM_TARGET_DESC

{'LMC': 'LMC',
 'SMC': 'SMC',
 'ORION': 'ORION',
 'ORION_SPARSE': 'sparse sampling of the surrounding orion region',
 'MW1': 'MW1',
 'MW2': 'MW2',
 'THOR': 'THOR',
 'FULLSKY': 'sparse sampling of the full sky'}

In [7]:
desi_template["obsconditions"]

[['DARK', 0, 'Moon is down'],
 ['GRAY', 1, 'Moon up; illum fraction<0.6 and (illum*elev)<30 deg'],
 ['BRIGHT', 2, 'Moon up and bright (not GRAY)'],
 ['BACKUP', 3, 'Very bad seeing, high extinction, or bright cloud cover'],
 ['TWILIGHT12', 4, 'Between sunset/sunrise and 12 degree twilight'],
 ['TWILIGHT18', 5, 'Between 12 and 18 degree twilight'],
 ['DAY', 6, 'Daytime calibrations'],
 ['CLOSED', 7, 'Nighttime but dome is closed due to rain, wind, dew...'],
 ['APOCALYPSE', 8, 'Conditions are so bad that the world is ending anyway'],
 ['IGNORE',
  9,
  'Used to indicate that obsconditions are set by a sub-bit rather than this bit']]

In [8]:
def lvm2desi_obsconditions(max_lunation, min_moon_dist, max_airmass=None, min_shadowhight=None, **kwargs):
    """Return observing conditions transformed from the LVM to the DESI definition"""
    if max_lunation == 0 or min_moon_dist >= 30:
        return "DARK"
    elif max_lunation < 0.6 and max_lunation*min_moon_dist < 30:
        return "GRAY"
    else:
        return "BRIGHT"

In [9]:
desi_template["obsmask"]

[['UNOBS', 0, 'unobserved'],
 ['OBS', 1, 'observed but no spectro pipeline answer yet'],
 ['DONE', 2, 'enough observations already obtained'],
 ['MORE_ZWARN', 3, 'ambiguous redshift; need more observations'],
 ['MORE_ZGOOD', 4, 'redshift known; need more observations'],
 ['MORE_MIDZQSO', 8, 'mid-z QSO; more observations at very low priority'],
 ['DONOTOBSERVE', 16, 'Do not observe this target (possibly temporarily)']]

In [10]:
LVM_TARGET_BITS = [
    [
        targetclass,
        targetbit,
        LVM_TARGET_DESC[targetclass],
        {
            "obsconditions": lvm2desi_obsconditions(**lvm_template[targetclass]),
            "priorities": {"UNOBS": lvm_template[targetclass]["priority"]}
        }
    ] for targetclass, targetbit in zip(lvm_template.keys(),range(len(lvm_template.keys())))]
LVM_TARGET_BITS

[['LMC', 0, 'LMC', {'obsconditions': 'DARK', 'priorities': {'UNOBS': 30}}],
 ['SMC', 1, 'SMC', {'obsconditions': 'DARK', 'priorities': {'UNOBS': 30}}],
 ['ORION', 2, 'ORION', {'obsconditions': 'DARK', 'priorities': {'UNOBS': 19}}],
 ['ORION_SPARSE',
  3,
  'sparse sampling of the surrounding orion region',
  {'obsconditions': 'DARK', 'priorities': {'UNOBS': 9}}],
 ['MW1', 4, 'MW1', {'obsconditions': 'DARK', 'priorities': {'UNOBS': 10}}],
 ['MW2', 5, 'MW2', {'obsconditions': 'DARK', 'priorities': {'UNOBS': 10}}],
 ['THOR', 6, 'THOR', {'obsconditions': 'DARK', 'priorities': {'UNOBS': 10}}],
 ['FULLSKY',
  7,
  'sparse sampling of the full sky',
  {'obsconditions': 'DARK', 'priorities': {'UNOBS': 1}}]]